In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time

In [3]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [5]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# URL of page to be scraped
url = 'https://npin.cdc.gov/search/organization/testing/hiv'
#adding in splinter needed item
browser.visit(url)

#Scrape by state search function

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)
print(response.text)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')
# Examine the results, then determine element that contains sought info
print(soup.prettify())

In [ ]:
# results are returned as an iterable list
results = soup.find_all('div', class_="ds-1col node node-organization article user-picture-available view-mode-search_result clearfix")
for r in results:
    print(r.prettify())

In [ ]:
organizations = []

#Iterate through all pages
for x in range(890):
    #HTML object
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html, 'html.parser')
    #Retrieve all elements that contain organization information in an iterable list
    results = soup.find_all('div', class_="ds-1col node node-organization article user-picture-available view-mode-search_result clearfix")
    
    #Iterate through each organization using Beautiful Soup's find methods
    for r in results:
        
        #returns organization name
        
        #returns organization name (first line)
        try:
            n_step = r.find(class_='field')
            n_step2 = n_step.find('a')
            name = n_step2.text
        except:
            name = ' '
        #returns organization name (second line)    
        try:
            n2_step = r.find(class_='field-name-field-organization-name-2')
            n2_step2 = n2_step.find('a')
            name2 = n2_step2.text
        except (NameError, AttributeError):
            name2 = ' '
        full_name = (name + ' ' + name2)
        
        #returns organization address
        try:
            a_step = r.find('section', class_='field')
            a_step2 = a_step.find(class_='field-item even')
            address = a_step2.text
        except:
            address = 'need info'
                    
        #returns organization phone number    
        try:
            p_step = r.find(class_='field-name-phone-number')
            p_step2 = p_step.find(class_='field-item even')
            phone_num = p_step2.text
        except:
            phone_num = 'need info'
            
        #place info into dictionary  
        organizations.append({'Name':full_name, 'Address':address, 'Phone Number':phone_num})
        
    #Click the 'next' button on each page
    #time.sleep(5)  # wait 5 seconds before clicking next button if program is going too fast for webpage
    try:
        browser.find_by_css('a.fsrDeclineButton').click()
    except:
        try:
            browser.find_by_css('li.pager-next a.action-item').click()
        except Exception as e:
            print(e)


In [ ]:
org_df = pd.DataFrame(organizations, columns=['Name', 'Address', 'Phone Number'])
org_df

In [ ]:
org_df['Address'] = org_df['Address'].str.replace(',', '')
org_df['Address'].head()

In [ ]:
# Export file as a CSV, with the Pandas index and headers
org_df.to_csv("hiv_providers.csv", index=True, header=True)